# 作るもの：知ってる人が来たら自動で扉を開ける

AIカメラの顔認識機能を使って、知っている人なら扉を自動で開けるプログラムを作ります。以下はシステムの全体です。

<img src='https://github.com/TomoniCodeAcademy/LEGO-expoert-course/blob/main/img/pictures/HuskyLens_facerecognition_auto_gate.png?raw=true' width=400>

自動とびらの動きは以下の動画を見てください。


知っている人と判断できるように、自分の顔をAIカメラにが学習させます。一度学習させると、カメラに顔を向けるとカメラは正しく認識してくれます。

まず、AIカメラを保持できるようホルダを作ります。

<img src='https://github.com/TomoniCodeAcademy/LEGO-expoert-course/blob/main/img/pictures/HuskyLens_facerecognition_camera_holder.png?raw=true' width=400>

# LEGOとAIカメラの接続
下記の写真のように、LEGOとAIカメラを接続してください。AIカメラを安定して動作させるため、AIカメラの電源は携帯バッテリーから供給します。LEGOのポートは [F] を使ってください。

# PythonによるAIカメラの操作

まずはPythonによりAIカメラの顔認識を行います。LEGO SPIKEから以下のプログラムを打ち込んでください。

In [ ]:
import sys
import time
from hub import port

sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)

上記プログラムはLEGO SPIKEに接続したAIカメラを使うための準備です。非常にややこしいのですが、今時点ではこういうものと思ってください。

- from huskylens_lib import HuskyLensという行がありますが、これはHuskyLensを使うための関数を取り込む作業をしています。
- port.F.mode(,...)という行は、LEGO SPIKE HUBの　FポートとAIカメラを接続するように指示しています。
- husky = HuskyLens(port.F) では AIカメラ(HuskyLens）を使えるように準備しています。

上記プログラムコードによりAIカメラを使う準備ができました。手でカメラを操作していた時は、ダイアルを動かして顔認識モードに切り替えをしていました。この作業はプログラムで行えます。以下のコードを最後の行に追加してください。

In [ ]:
husky.send_CMD_REQ_ALGO(Algo.FACE_RECOGNITION)

上記関数を実行すると、AIカメラが顔認識モードに切り替わります。

以上の作業によりAIカメラが顔認識モードで動作可能になりました。AIカメラとLEGOを組み合わせるには、AIカメラに対して、今誰かカメラに写っているか？問い合わせる必要があります。問い合わせるための関数も用意されています。

In [ ]:
husky.read_blocks()

上記関数を実行すると、カメラに学習ずみの顔が映っているかどうかを返却します。もし学習済みの顔が映っていると以下と返却します。

In [ ]:
>>> husky.read_blocks()
[('block', 1, 164, 141, 109, 146)]

もしだれの顔も映っていない場合は以下と返却します。空のリストを返却します。

In [ ]:
>>> husky.read_blocks()
[]

もし顔が映っているが知らない人の場合、リストの中のリストの２番目の値が0となります。これはID:0であることを示しています。

In [ ]:
[('block', 0, 137, 140, 48, 82)]

では、一度だけではなくて何度も繰り返し確認するプログラムに修正しましょう。プログラム全文は以下となります。

In [ ]:
import sys
import time
from hub import port

sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)        # AIカメラ初期化
husky.send_CMD_REQ_ALGO(Algo.FACE_RECOGNITION)  # 顔認識モードに設定

while True:
    print(husky.read_blocks())   # 認識状態を取得、表示

上記プログラムを実行して画面に表示される文字列を確認してください。
学習した人の顔を映した時、学習していない顔を映した時で表示が変わるのを確認してください

In [ ]:
カメラに顔が映っていない場合は以下の表示となります。
```
[]
[]
[]
[]
```
学習した人の顔が映ると以下の表示に切り替わります。
```
[('block', 1, 164, 141, 109, 146)]
[('block', 1, 164, 141, 109, 146)]
```
学習していない場合の表示は以下となります。
```
[('block', 0, 137, 140, 48, 82)]
[('block', 0, 137, 140, 48, 82)]
```

このように、リストの中の２番目に、対象のIDが入っています。学習した顔の場合、２番目の値が１となります。

# 知っている顔を映したら音を流しましょう

覚えている人の顔がカメラに映ったら　関数 husky.read_blocks()　が返却するリストの中のIDが１　になります。if  文で、IDが　１かどうかを判断して、1 だったら音を鳴らします。  先ほどのプログラムを改良して音がなるように変更しましょう。

In [ ]:
import sys
import time
from hub import port

sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens
from spike import PrimeHub

hub = PrimeHub()           # HUBの初期化


port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)        # AIカメラ初期化
husky.send_CMD_REQ_ALGO(Algo.FACE_RECOGNITION)  # 顔認識モードに設定
time.sleep(1)

while True:
    target = husky.read_blocks()   # 認識状態を取得、表示
    if len(target) == 0:           # 何も映っていない場合
        print('no face')           # 処理しない
    else:                          # 顔が映っている場合
        print(target)
        block, face_id, x, y, w, h = target[0]  # 値を取り出す
        if face_id == 1:            # face id が1 の場合
            print('learned face')
            hub.speaker.beep(note=80, seconds=0.2)
            time.sleep(1)           # 繰り返して鳴るのを防ぐ
        else:
            print('unknown face')


以上で顔を学習して、学習した顔がAIカメラに映ったら音が鳴るプログラムができました。これを改良して学習した顔で扉が開くプログラムに変更したいと思います